<img src="https://raw.githubusercontent.com/maverick-faus/Files/master/DeepDeamon2.png" alt="drawing" width="100px"/>
# Red BNB - v1.2
TT: Deep Deep Dishes  
Food recommendation System  
ESCOM  
By Faus - 2K18


**Instalación de TensorBoard para Collaboratory**

In [1]:
!pip install -U tensorboardcolab
from tensorboardcolab import *

  Running setup.py bdist_wheel for tensorboardcolab ... - done
  Stored in directory: /root/.cache/pip/wheels/ab/74/02/cda602d1dc28b2f12eab313c49b9bfa14d6371326bc2590e06
Successfully built tensorboardcolab


Using TensorFlow backend.


**Instalación de biblioteca para acceso a Google Drive**

In [2]:
!pip install PyDrive
import os
from zipfile import ZipFile
from shutil import copy
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


    100% |████████████████████████████████| 993kB 19.5MB/s 
  Running setup.py bdist_wheel for PyDrive ... - done
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built PyDrive


**Autenticación de usuario de Google Drive**

In [3]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package libfuse2:amd64.
(Reading database ... 22298 files and directories currently installed.)
Preparing to unpack .../libfuse2_2.9.7-1ubuntu1_amd64.deb ...
Unpacking libfuse2:amd64 (2.9.7-1ubuntu1) ...
Selecting previously unselected package fuse.
Preparing to unpack .../fuse_2.9.7-1ubuntu1_amd64.deb ...
Unpacking fuse (2.9.7-1ubuntu1) ...
Selecting previously unselected package google-drive-ocamlfuse.
Preparing to unpack .../google-drive-ocamlfuse_0.7.0-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.0-0ubuntu1~ubuntu18.04.1) ...
Setting up libfuse2:amd64 (2.9.7-1ubuntu1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
Setting up fuse (2.9.7-1ubuntu1) ...
Setting up google-drive-ocamlfuse (0.7.0-0ubuntu1~ubuntu18.04.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleus

**Agregar Drive como directorio**

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
!ls drive

'Advanced Neural Networks'
'Artículos Varios TT'
'BBVA Hack - Chicos que lloran'
'Books Wishlist 2K18.ods'
'Books Wishlist 2K18.xlsx'
'Books Wishlist 2K18.xlsx.ods'
 Chatboot
 Classroom
 CodigoTT
'COMUNICADO Patish Complemento de Pago a sus proveedores.docx'
'COMUNICADO Patish Complemento de Pago a sus proveedores.odt'
'Copia de Red4B.ipynb'
'Curso DNN'
'Deep Daemon'
 deepdeep.backup
'Discurso excelencia.odt'
'Documento sin título.odt'
'Documentos Personales '
'Documento TT1'
'Documento TT1 (3a7c594f)'
 DriveC
'Hoja de cálculo sin título.ods'
 Imagenet_Hamburguer.ods
 IMG_2364Trim.mp4
'Prueba de Escritorio - DDD.ods'
'Publicidad dirigida mediante redes neuronales profundas.pdf'
'Retorno Inversiones Jose Shedid Merhy.xlsx'
'Retorno Inversiones Jose Shedid Merhy.xlsx.ods'
'Script Presentación TT1.odt'
'TT1 - Previos'
 TT2
 TT2_W2VReporteFinalFinal.docx
 Untitled0.ipynb


In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
fileIdLabelmap = '1_2gMy6MBqSiqMM1ybj3obdsrVMFEPezX'
#label map
downloaded1 = drive.CreateFile({'id': fileIdLabelmap})
downloaded1.GetContentFile("BNB_train.pickle")

In [0]:
!ls


adc.json  BNB_train.pickle  drive  sample_data


In [0]:
import tensorflow as tf
import numpy as np
import pickle 
from sklearn.metrics import confusion_matrix

In [0]:
def conv2d(x, W,name,padd,strid=[1,1,1,1]):
    #El stride de esa función no reduce el tamaño de la imagen
    return tf.nn.conv2d(x, W, strides=strid, padding=padd,name=name)

def maxpool2d(x,ks,st):
    #           El st de esta función reduce la imagen a la mitad
    return tf.nn.max_pool(x, ksize=ks, strides=st, padding='SAME')

In [0]:
def reset_graph():
    #Limpiamos la gráfic
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()

In [0]:
def deep_neural_convolutional_class(
    batch_size=20,
    image_size=[200,200],
    Drop_prob=1.0,
    learning_rate = 1e-3,
    n_nodes_hl0 = 2000,
    n_nodes_hl1 = 1000,
    n_nodes_hl2 = 500,
    n_nodes_hl3 = 100,
    n_classes=2
    ):
    
    reset_graph()
    #Place holder de entrada 
    x= tf.placeholder(tf.float32,[batch_size,image_size[0],image_size[1],3], name='placeholder_img_entrada')
    y=tf.placeholder(tf.float32,name='placeholder_one_hot')
  
    #Diccionario de pesos convolucionales 
    with tf.name_scope('pesos_bias') as scope1:
        weigths={"w_conv1":tf.Variable(tf.random_normal([5,5,3,32]),name='Pesos_1_32'),
                 "w_conv2":tf.Variable(tf.random_normal([5,5,32,64]),name='Pesos_1_64'),
                 "w_conv3":tf.Variable(tf.random_normal([3,3,64,128]),name='Pesos_1_128'),   
                 "w_conv4":tf.Variable(tf.random_normal([5,5,128,256]),name='Pesos_1_256'),
                }
        #Diccionario de bias
        biases={"b_conv1":tf.Variable(tf.random_normal([32]),name='Bias_1_32'),
                "b_conv2":tf.Variable(tf.random_normal([64]),name='Bias_1_64'),
                "b_conv3":tf.Variable(tf.random_normal([128]),name='Bias_1_128'),
                "b_conv4":tf.Variable(tf.random_normal([256]),name='Bias_1_256'),
               }

    #Extractor de características
    with tf.name_scope('capas_conv') as scope2:
        conv1=tf.nn.relu(conv2d(x,weigths["w_conv1"],'Capa_Conv_1','SAME')+biases["b_conv1"],name='Func_relu_1')
        conv1=tf.nn.dropout(conv1,Drop_prob)
        conv1=maxpool2d(conv1,ks=[1,2,2,1],st=[1,2,2,1])
        #imagen resultante de 100x100x32
        print(conv1)

        conv2=tf.nn.relu(conv2d(conv1,weigths["w_conv2"],'Capa_Conv_2','SAME')+biases["b_conv2"],name='Func_relu_2')
        conv2=tf.nn.dropout(conv2,Drop_prob)
        conv2=maxpool2d(conv2,ks=[1,2,2,1],st=[1,2,2,1])
        #imagen resultante de 50x50x64
        print(conv2)

        conv3=tf.nn.relu(conv2d(conv2,weigths["w_conv3"],'Capa_Conv_3','VALID')+biases["b_conv3"],name='Func_relu_3')
        conv3=tf.nn.dropout(conv3,Drop_prob)
        conv3=maxpool2d(conv3,ks=[1,2,2,1],st=[1,2,2,1])
        #imagen resultante de 24x24x128
        print(conv3)

        conv4=tf.nn.relu(conv2d(conv3,weigths["w_conv4"],'Capa_Conv_4','SAME')+biases["b_conv4"],name='Func_relu_4')
        conv4=tf.nn.dropout(conv4,Drop_prob)
        conv4=maxpool2d(conv4,ks=[1,2,2,1],st=[1,2,2,1])
        #imagen resultante de 12x12x256
        print(conv4)

        #Embeding, son las caracteristicas fonales que se pasarán al MLP o red completamente conectada para clasifiacar
        embdeding=tf.reshape(conv4,[batch_size,12*12*256],name='Embeding')
        print(embdeding)
    
    #Red perceptron, declaración de capas, son diccionarios de pesos y bias.
    with tf.name_scope('capas_clasificador') as scope3:
        hidden_0_layer = {'weights':tf.Variable(tf.random_normal([12*12*256, n_nodes_hl0]),name='Capa_oculta_pesos_0'),
                          'biases':tf.Variable(tf.random_normal([n_nodes_hl0]),name='Capa_oculta_bias_0')}

        hidden_1_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl0, n_nodes_hl1]),'Capa_oculta_pesos_1'),
                          'biases':tf.Variable(tf.random_normal([n_nodes_hl1]),name='Capa_oculta_bias_1')}

        hidden_2_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2]),'Capa_oculta_pesos_2'),
                          'biases':tf.Variable(tf.random_normal([n_nodes_hl2]),name='Capa_oculta_bias_2')}
        
        hidden_3_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3]),'Capa_oculta_pesos_3'),
                          'biases':tf.Variable(tf.random_normal([n_nodes_hl3]),name='Capa_oculta_bias_3')}

        output_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl3, n_classes]),'Capa_salida_pesos'),
                        'biases':tf.Variable(tf.random_normal([n_classes]),name='Capa_salida_bias'),}
    
    #W*P + B 
    with tf.name_scope('op_clasificador') as scope4:
        
        l0 = tf.add(tf.matmul(embdeding,hidden_0_layer['weights'],name='Matmul_l0'), hidden_0_layer['biases'],name='Suma_Pesos_Bias_0')
        l0 = tf.nn.relu(l0,name='l0_relu_0')

        l1 = tf.add(tf.matmul(l0,hidden_1_layer['weights'],name='Matmul_l1'), hidden_1_layer['biases'],name='Suma_Pesos_Bias_1')
        l1 = tf.nn.relu(l1,name='l1_relu_1')

        l2 = tf.add(tf.matmul(l1,hidden_2_layer['weights'],name='Matmul_l2'), hidden_2_layer['biases'],name='Suma_Pesos_Bias_2')
        l2 = tf.nn.relu(l2,name='l2_relu_2')
        
        l3 = tf.add(tf.matmul(l2,hidden_3_layer['weights'],name='Matmul_l3'), hidden_3_layer['biases'],name='Suma_Pesos_Bias_3')
        l3 = tf.nn.relu(l3,name='l3_relu_3')

        output = tf.matmul(l3,output_layer['weights'],name='Matmul_out') + output_layer['biases']
    
    # Declarando la funcion de costo y entrenamiento
    #Reduce mean, reduce la dimension del tensor en un promedio es decir hace el promedio del costo o error
    
    
    with tf.name_scope('costo_y_optimizador') as scope5:
        cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=output,labels=y,name='softmax_cross_entropy_with_logits1'),name='reduce_mean1')
        optimizer = tf.train.AdamOptimizer(name='Adam',learning_rate=learning_rate).minimize(cost,name='minimo')
    
    
    tf.summary.scalar("costo",cost)
    correct = tf.equal(tf.argmax(output,1),tf.argmax(y,1),name='valores_correctos')
    accuracy = tf.reduce_mean(tf.cast(correct,'float'),name='Promedio_exactitud') #porcentaje de error
    
    tf.summary.scalar("accuracy",accuracy)
    summaries = tf.summary.merge_all()
    
    return dict(
        x = x,
        y=y,
        embeding=conv4,
        output=output,
        saver = tf.train.Saver(),
        total_loss = cost,
        train_step = optimizer,
        summaries = summaries,
        accuracy = accuracy
    )

In [11]:
deep_neural_convolutional_class()

Tensor("capas_conv/MaxPool:0", shape=(20, 100, 100, 32), dtype=float32)
Tensor("capas_conv/MaxPool_1:0", shape=(20, 50, 50, 64), dtype=float32)
Tensor("capas_conv/MaxPool_2:0", shape=(20, 24, 24, 128), dtype=float32)
Tensor("capas_conv/MaxPool_3:0", shape=(20, 12, 12, 256), dtype=float32)
Tensor("capas_conv/Embeding:0", shape=(20, 36864), dtype=float32)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



{'accuracy': <tf.Tensor 'Promedio_exactitud:0' shape=() dtype=float32>,
 'embeding': <tf.Tensor 'capas_conv/MaxPool_3:0' shape=(20, 12, 12, 256) dtype=float32>,
 'output': <tf.Tensor 'op_clasificador/add:0' shape=(20, 2) dtype=float32>,
 'saver': <tensorflow.python.training.saver.Saver at 0x7f5d1bcf8b38>,
 'summaries': <tf.Tensor 'Merge/MergeSummary:0' shape=() dtype=string>,
 'total_loss': <tf.Tensor 'costo_y_optimizador/reduce_mean1:0' shape=() dtype=float32>,
 'train_step': <tf.Operation 'costo_y_optimizador/minimo' type=NoOp>,
 'x': <tf.Tensor 'placeholder_img_entrada:0' shape=(20, 200, 200, 3) dtype=float32>,
 'y': <tf.Tensor 'placeholder_one_hot:0' shape=<unknown> dtype=float32>}

In [0]:
def train_img_encoder(G,nameBatch,numbatch, num_epochs,learning_rate = 1e-2, 
                      num_steps = 15, batch_size = 20, verbose = True, save=False,checkpoint=False):
    tf.set_random_seed(2345)
    tf.logging.set_verbosity(tf.logging.ERROR)
    print('Start training')
    total_loss = 0
    with tf.Session() as sess:
        
        writer = tf.summary.FileWriter("./CNNClass")
        tf.summary.FileWriter.add_graph(writer,sess.graph)
        
        sess.run(tf.global_variables_initializer()) ###########
        #print('Enter here4')
        if checkpoint:
            ENCname="./"+checkpoint+".ckpt"
            G['saver'].restore(sess, ENCname)
        training_losses = []
        for epoch in range(num_epochs):
            #print('ok')
            epoch_loss = 0
            for key in range(numbatch):
                data=np.array(pickle.load(open(nameBatch+".pickle","rb")))
                dim,var=data.shape
                for j in range(int(dim/batch_size)):
                    epoch_x=data[batch_size*(j):batch_size*(j+1),0].tolist()
                    epoch_Y=data[batch_size*(j):batch_size*(j+1),1].tolist()

                    feed_dict={G['x']: epoch_x,
                               G['y']: epoch_Y,
                              }
                
                    total_loss,_,summ,accuracy = sess.run([G["total_loss"],G["train_step"],G["summaries"],G["accuracy"]],feed_dict)

                epoch_loss += total_loss
                writer.add_summary(summ,epoch)
                tbc.save_value("Loss", "loss", epoch, epoch_loss)
                tbc.save_value("Accuracy", "accuracy", epoch, accuracy)
                tbc.flush_line("loss")
                tbc.flush_line("accuracy")
            if verbose:
                print("Average training loss for Epoch", epoch, ":", epoch_loss)
            training_losses.append(epoch_loss)
      
        if isinstance(save, str):
            ENCname="drive/CodigoTT/BNB/ckptBNB/"+save+".ckpt"
            G['saver'].save(sess, ENCname)       
        tbc.close()
    return training_losses

In [13]:
tbc=TensorBoardColab()

Wait for 8 seconds...
TensorBoard link:
http://9be2bb1f.ngrok.io


In [0]:
path = "BNB_train"
encoder=deep_neural_convolutional_class(batch_size = 20)
train_img_encoder(encoder,path,1,num_epochs=15,save="BNB1",batch_size = 20,verbose = True)

Tensor("capas_conv/MaxPool:0", shape=(20, 100, 100, 32), dtype=float32)
Tensor("capas_conv/MaxPool_1:0", shape=(20, 50, 50, 64), dtype=float32)
Tensor("capas_conv/MaxPool_2:0", shape=(20, 24, 24, 128), dtype=float32)
Tensor("capas_conv/MaxPool_3:0", shape=(20, 12, 12, 256), dtype=float32)
Tensor("capas_conv/Embeding:0", shape=(20, 36864), dtype=float32)
Start training
Average training loss for Epoch 0 : 5597262512128.0
Average training loss for Epoch 1 : 1060663328768.0
Average training loss for Epoch 2 : 86806994944.0
Average training loss for Epoch 3 : 484335747072.0
Average training loss for Epoch 4 : 2656910442496.0
Average training loss for Epoch 5 : 117250244608.0
Average training loss for Epoch 6 : 560080093184.0
Average training loss for Epoch 7 : 177584963584.0
Average training loss for Epoch 8 : 62041120768.0
Average training loss for Epoch 9 : 0.0
Average training loss for Epoch 10 : 32517154816.0
